In [1]:
import scanpy as sc
import pandas as pd
import numpy as np

In [9]:
adata = sc.read("/Users/kang/Dropbox/Code/data/batch2_all_normalized_filtered.h5ad")

In [10]:
adata

AnnData object with n_obs × n_vars = 24673 × 35635
    obs: 'tsne1', 'tsne2', 'ind', 'stim', 'cluster', 'cell', 'multiplets'
    var: 'gene symbol'
    uns: 'log1p'

In [19]:
sc.tl.rank_genes_groups(adata, groupby = "cell", pts = True)

In [4]:
adata.uns["rank_genes_groups"].keys()

dict_keys(['params', 'pts', 'pts_rest', 'names', 'scores', 'pvals', 'pvals_adj', 'logfoldchanges'])

In [16]:
df_deg = format_DEGs(adata)

In [13]:
def format_DEGs(adata):
    keys = ["names","scores","logfoldchanges","pvals","pvals_adj","pts","pts_rest"]
    for i,key in enumerate(keys):
        a = pd.DataFrame(adata.uns["rank_genes_groups"][key]) # transfer to data frame
        b = pd.DataFrame(a.values.T.reshape(1,a.shape[0]*a.shape[1]).T) # reformat the data frame to one column
           
        if i == 0:
            b.columns = [key] # rename the column name
            b["Status"] = sorted(list(a.columns)*a.shape[0]) # add Status annotation
            b.set_index([key],inplace=True)
            b_merged = b
        else:
            if key in ["pts","pts_rest"]:
                pts_all = []
                for cell_group in np.unique(b_merged["Status"]):
                    genes = b_merged.loc[b_merged["Status"] == cell_group,:].index.values
                    pts_all = pts_all + list(a.loc[genes, cell_group])
                b_merged[key] = pts_all
            else:
                b_merged[key] = list(b[0])
        
    return b_merged

In [17]:
df_deg

,Status,scores,logfoldchanges,pvals,pvals_adj,pts,pts_rest
names,,,,,,,
CD74,B cells,151.864487,7.343440,0.0,0.0,0.990284,0.526109
HLA-DRA,B cells,130.348312,7.762211,0.0,0.0,0.971240,0.368371
HLA-DRB1,B cells,76.798622,6.471466,0.0,0.0,0.881461,0.337828
CD79A,B cells,73.298073,10.871271,0.0,0.0,0.690245,0.012670
HLA-DPA1,B cells,62.462658,5.916356,0.0,0.0,0.806840,0.295339
...,...,...,...,...,...,...,...
KYNU,NK cells,-64.092697,-5.744125,0.0,0.0,0.013046,0.261596
HLA-DRA,NK cells,-66.111351,-6.009887,0.0,0.0,0.071249,0.462875
LTB,NK cells,-66.117226,-5.653219,0.0,0.0,0.035625,0.376014


In [18]:
df_deg.to_csv("/Users/kang/Dropbox/Code/data/toppcell_test/DEGs_cell.txt", sep = "\t")

In [25]:
a = pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(5)
a

,B cells,CD4 T cells,CD8 T cells,CD14+ Monocytes,Dendritic cells,FCGR3A+ Monocytes,Megakaryocytes,NK cells
0,CD74,GIMAP7,CCL5,TYROBP,CD74,TYROBP,PPBP,NKG7
1,HLA-DRA,LTB,NKG7,C15orf48,HLA-DRA,TIMP1,GNG11,GNLY
2,HLA-DRB1,CCR7,APOBEC3G,FCER1G,HLA-DRB1,FCER1G,PF4,GZMB
3,CD79A,LDHB,CST7,CD63,HLA-DPA1,TYMP,SDPR,CCL5
4,HLA-DPA1,CD3D,B2M,TIMP1,HLA-DPB1,S100A11,NCOA4,APOBEC3G
